In [1]:
# Import MNIST dataset from torchvision
import torch
import torchvision
from torch.utils.data import random_split, DataLoader, TensorDataset
import torch.nn as nn

# Load Data
mnist_dataset = torchvision.datasets.MNIST(root = ".", download = True)

train_size = int(0.9 * len(mnist_dataset))
val_size = len(mnist_dataset) - train_size

# Train / Val Split
train_dataset, val_dataset = random_split(mnist_dataset, [train_size, val_size])

train_x_data = train_dataset.dataset.data[train_dataset.indices]
train_y_data = train_dataset.dataset.targets[train_dataset.indices]
val_x_data   = val_dataset.dataset.data[val_dataset.indices]
val_y_data   = val_dataset.dataset.targets[val_dataset.indices]

# Comb TensorDataset
train_tensor_dataset = TensorDataset(train_x_data, train_y_data)
val_tensor_dataset = TensorDataset(val_x_data, val_y_data)

# DataLoader
train_loader = DataLoader(train_tensor_dataset, batch_size=128, shuffle=True, num_workers=0)
val_loader = DataLoader(val_tensor_dataset, batch_size=128, shuffle=False, num_workers=0)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 505kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.98MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.92MB/s]


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
  # Your Model is Defined Here
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.Conv2d(1, 6, kernel_size=5, padding=2),
        nn.Sigmoid(),
        nn.AvgPool2d(kernel_size=2, stride=2),
        nn.Conv2d(6, 16, kernel_size=5),
        nn.Sigmoid(),
        nn.AvgPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(16 * 5 * 5, 120),
        nn.Linear(120, 84),
        nn.Linear(84,10)
    )

  def forward(self, x):
    return self.net(x)

model = LeNet()

In [9]:
from tqdm import tqdm

max_epochs = 20
learning_rate = 5 * 1e-5
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in tqdm(range(max_epochs)):
    model.train()
    total = 0
    correct = 0
    cur_loss = 0.0

    for train_x, train_y in train_loader:
        train_x = train_x.float().to(device)       # [batch, 28, 28]
        train_y = train_y.to(device)               # [batch]
        if train_x.dim() == 3:
            train_x = train_x.unsqueeze(1)         # [batch, 1, 28, 28]

        predict_y = model(train_x)                 # [batch, 10]
        loss = criterion(predict_y, train_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cur_loss += loss.item() * train_x.size(0)  # total loss
        preds = predict_y.argmax(dim=1)            # [batch]
        correct += (preds == train_y).sum().item()
        total += train_x.size(0)

    avg_loss = cur_loss / total
    acc = correct / total

    val_total = 0
    val_correct = 0
    for val_x, val_y in val_loader:
      val_x = val_x.float().to(device)
      val_y = val_y.to(device)
      if val_x.dim() == 3:
          val_x = val_x.unsqueeze(1)         # [batch, 1, 28, 28]
      predict_val_y = model(val_x)                 # [batch, 10]

      preds = predict_val_y.argmax(dim=1)            # [batch]
      val_correct += (preds == val_y).sum().item()
      val_total += val_x.size(0)

    val_acc = val_correct / val_total
    print(f"Epoch {epoch}: Loss={avg_loss:.4f}, Acc={acc:.4f}, Val_Acc={val_acc:.4f}")


  3%|▎         | 1/30 [00:01<00:43,  1.49s/it]

Epoch 0: Loss=0.1045, Acc=0.9691, Val_Acc=0.9657


  7%|▋         | 2/30 [00:02<00:40,  1.46s/it]

Epoch 1: Loss=0.1029, Acc=0.9694, Val_Acc=0.9668


 10%|█         | 3/30 [00:04<00:39,  1.45s/it]

...

Epoch 27: Loss=0.0800, Acc=0.9761, Val_Acc=0.9712


 97%|█████████▋| 29/30 [00:44<00:01,  1.58s/it]

Epoch 28: Loss=0.0791, Acc=0.9765, Val_Acc=0.9722


100%|██████████| 30/30 [00:45<00:00,  1.53s/it]

Epoch 29: Loss=0.0785, Acc=0.9768, Val_Acc=0.9728
